In [ ]:
# Third-party
import astropy.coordinates as coord
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as pl
import numpy as np
pl.style.use('apw-notebook')
%matplotlib inline
from scipy.stats import powerlaw, pareto
from scipy.linalg import expm3

import gary.coordinates as gc
import gary.dynamics as gd
import gary.integrate as gi
import gary.potential as gp

import ophiuchus.potential as op
_p = op.load_potential('static_mw')
pars = _p.parameters.copy()
potential = op.OphiuchusPotential(**pars)

Taking parameters from the anisotropic model in Gnedin & Ostriker 1997 (§2.2):
$$
\frac{\sigma_r^2}{\sigma_t^2} = 1 + \frac{r^2}{r_a^2}\\
\rho \propto r^{-\alpha}\\
\sigma_r^2 = (v_{\rm circ}^2 - v_{\rm rot}^2)\frac{r^2/(\alpha - 2) + r_a^2/\alpha}{r^2 + r_a^2}\\
f(v_r, v_t) = A\exp{\left(-\frac{v_r^2}{2\sigma_r^2}-\frac{v_t^2}{2\sigma_t^2} \right)}
$$
with
$$
v_{\rm circ} = 220~{\rm km}~{\rm s}^{-1}\\
v_{\rm rot} = 60~{\rm km}~{\rm s}^{-1}\\
\alpha = 3
$$

The scheme is:
1. sample a radius
2. sample a random pair of angles (uniform on surface of sphere)
3. sample a radial velocity and peculiar tangential velocity
4. sample a new random pair of angles as the position angle for the tangential velocity
5. add the rotation velocity to the in-plane component of the peculiar tangential velocity

In [ ]:
def vcirc(r):
    q = np.zeros((3,len(r)))
    q[0] = r
    vcx = np.sqrt(potential.G * potential.mass_enclosed(q) / r)
    
    q = np.zeros((3,len(r)))
    q[1] = r
    vcy = np.sqrt(potential.G * potential.mass_enclosed(q) / r)
    
    q = np.zeros((3,len(r)))
    q[2] = r
    vcz = np.sqrt(potential.G * potential.mass_enclosed(q) / r)
    
    return np.mean([vcx,vcy,vcz], axis=0)

In [ ]:
# vcirc = 220. # km/s
vrot = 60. # km/s
alpha = 3.
ra = 7.6 # kpc

## Position

In [ ]:
# r = powerlaw.rvs(size=10, a=-3.)
r = np.logspace(-1, 2.5, 1024)

pl.loglog(r, pareto.pdf(r, b=2), marker=None)
pl.loglog(r, r**-3, marker=None)

In [ ]:
r = pareto.rvs(b=2, size=100000)
r = r[r>=10][:1000]
pl.hist(r, bins=np.logspace(0,2.5,32))
pl.xscale('log')
pl.yscale('log')

In [ ]:
phi = np.random.uniform(0, 2*np.pi, size=r.size)
theta = np.arccos(2*np.random.uniform(size=r.size) - 1)

In [ ]:
pos = coord.PhysicsSphericalRepresentation(r=r*u.kpc, phi=phi*u.radian, theta=theta*u.radian)
xyz = pos.represent_as(coord.CartesianRepresentation).xyz

## Velocity

In [ ]:
_vc = (vcirc(r)*u.kpc/u.Myr).to(u.km/u.s).value
σr = np.sqrt((_vc**2 - vrot**2)*(r**2/(alpha-2) + ra**2/alpha) / (r**2 + ra**2))
vr = np.random.normal(0, σr)
pl.hist(vr);

In [ ]:
σt = σr/np.sqrt(1 + r**2/ra**2) 
vt = np.random.normal(0, σt)
pl.hist(vt)

In [ ]:
vtot = np.sqrt(vr**2 + vt**2)
pl.hist(vtot, bins=16);

In [ ]:
def get_random_velocity(xyz, vr, vt, vrot):
    for v in [vr, vt, vrot]:
        if not hasattr(v, 'unit'):
            raise TypeError("Input velocities must be Astropy Quantity objects.")
        
    n = xyz.shape[1]
    pos = coord.CartesianRepresentation(xyz*u.kpc)
    
    sph = pos.represent_as(coord.PhysicsSphericalRepresentation)
    _phi = sph.phi
    _the = sph.theta
    
    R1 = np.zeros((3,3,n))
    R2 = np.zeros((3,3,n))
    
    for i in range(n):
        R1[...,i] = coord.angles.rotation_matrix(-_phi[i], axis='z')
        R2[...,i] = coord.angles.rotation_matrix(-_the[i], axis='y')
    R = R1*R2
        
    # generate random unit vectors in the x-y plane to get a random tangential velocity
    rand_phi = np.random.uniform(0,2*np.pi,size=n)
    rand_xyz = coord.UnitSphericalRepresentation(lon=rand_phi*u.radian, 
                                                 lat=np.zeros_like(rand_phi)*u.radian)
    rand_xyz = rand_xyz.represent_as(coord.CartesianRepresentation).xyz.value
        
    # tangential velocity
    vt_xyz = vt * np.einsum('ijn,jn->in', R, rand_xyz)
    
    # radial velocity and rotation velocity in xyz
    _vr = vr.to(u.km/u.s).value
    _vrot = np.zeros_like(_vr) + vrot.to(u.km/u.s).value
    _tmp = np.zeros_like(_vr)
    vr_rot_xyz = gc.spherical_to_cartesian(pos, np.vstack([_vr,_vrot,_tmp])*u.km/u.s)
    
    return vt + vr_rot_xyz

vxyz = get_random_velocity(xyz, vr=vr*u.km/u.s, vt=vt*u.km/u.s, vrot=vrot*u.km/u.s)

In [ ]:
pl.hist(np.linalg.norm(vxyz, axis=0));

---

In [ ]:
w0 = gd.CartesianPhaseSpacePosition(pos=xyz, vel=vxyz)

In [ ]:
fig = w0.plot(subplots_kwargs=dict(figsize=(16,5)))

for ax in fig.axes:
    ax.set_xlim(-100,100)
    ax.set_ylim(-100,100)

In [ ]:
orbit = potential.integrate_orbit(w0, dt=2., nsteps=4000)

In [ ]:
orbit_sph,_ = orbit.represent_as(coord.PhysicsSphericalRepresentation)

In [ ]:
bins = np.logspace(0,2.5,32)
pl.hist(orbit_sph.r[0].value, bins=bins, alpha=0.5)
pl.hist(orbit_sph.r[-1].value, bins=bins, alpha=0.5)
pl.xscale('log')
pl.yscale('log')

In [ ]:
peri = u.Quantity([orbit[:,i].pericenter() for i in range(orbit.shape[1])])
apoc = u.Quantity([orbit[:,i].apocenter() for i in range(orbit.shape[1])])

peri = peri[apoc < 250*u.kpc]
apoc = apoc[apoc < 250*u.kpc]

In [ ]:
bins = np.logspace(-1,3,32)

pl.hist(peri[np.isfinite(peri)], bins=bins, alpha=0.5)
pl.hist(apoc[np.isfinite(apoc)], bins=bins, alpha=0.5)

pl.xscale('log')
pl.yscale('log')

In [ ]:
ecc = (apoc - peri) / (apoc + peri)
pl.hist(ecc[np.isfinite(ecc)], bins=np.linspace(0,1,10))

---

## More precise integration and frequencies

In [ ]:
import superfreq

In [ ]:
i = 1
dt,nsteps = gd.estimate_dt_nsteps(w0[i], potential, nperiods=256, nsteps_per_period=128)

In [ ]:
orbit = potential.integrate_orbit(w0[0], dt=dt, nsteps=nsteps, 
                                  Integrator=gi.DOPRI853Integrator)

In [ ]:
sf = superfreq.SuperFreq(orbit.t.value)

In [ ]:
fs = [orbit.pos[i].decompose(potential.units).value + 1j*orbit.vel[i].decompose(potential.units).value 
      for i in range(3)]
freqs,_,_ = sf.find_fundamental_frequencies(fs)
superfreq.closest_resonance(np.abs(freqs), max_int=30)